In [2]:
import numpy as np
import folium
import plotly.graph_objects as go
from pyproj import Transformer

# ===== INPUTS =====
observer_lat = 31.963158
observer_lon = 35.930359
observer_alt = 2  # meters

pitch_deg = -10   # camera pitch (negative = down)
yaw_deg = 45      # yaw degrees from North (0 = North, 90 = East)
target_distance = 50  # meters (max distance to tank)

# ===== COMPUTE DIRECTION VECTOR (ENU) =====
pitch = np.radians(pitch_deg)
yaw = np.radians(yaw_deg)

dx = np.cos(pitch) * np.sin(yaw)   # East component
dy = np.cos(pitch) * np.cos(yaw)   # North component
dz = np.sin(pitch)                  # Up component

direction_vector = np.array([dx, dy, dz])
direction_vector /= np.linalg.norm(direction_vector)

# Scale so that line intersects ground (up=0)
scale = -observer_alt / dz
scale = min(scale, target_distance)

enu_target = direction_vector * scale
east_offset, north_offset, up_offset = enu_target

# ===== COORDINATE TRANSFORMATIONS =====
ecef_from_llh = Transformer.from_crs("epsg:4326", "epsg:4978", always_xy=True)
llh_from_ecef = Transformer.from_crs("epsg:4978", "epsg:4326", always_xy=True)

x0, y0, z0 = ecef_from_llh.transform(observer_lon, observer_lat, observer_alt)
origin_ecef = np.array([x0, y0, z0])

lat_rad = np.radians(observer_lat)
lon_rad = np.radians(observer_lon)

east = np.array([-np.sin(lon_rad), np.cos(lon_rad), 0])
north = np.array([
    -np.sin(lat_rad)*np.cos(lon_rad),
    -np.sin(lat_rad)*np.sin(lon_rad),
    np.cos(lat_rad)
])
up = np.array([
    np.cos(lat_rad)*np.cos(lon_rad),
    np.cos(lat_rad)*np.sin(lon_rad),
    np.sin(lat_rad)
])

ecef_target = origin_ecef + east_offset*east + north_offset*north + up_offset*up
target_lon, target_lat, target_alt = llh_from_ecef.transform(*ecef_target)

print(f"Estimated tank position: lat={target_lat:.6f}, lon={target_lon:.6f}, alt={target_alt:.2f}")

# ===== CREATE FOLIUM MAP =====
m = folium.Map(location=[observer_lat, observer_lon], zoom_start=17)

folium.Marker(
    [observer_lat, observer_lon],
    popup="Camera",
    icon=folium.Icon(color='blue')
).add_to(m)

folium.Marker(
    [target_lat, target_lon],
    popup="Tank",
    icon=folium.Icon(color='red')
).add_to(m)

folium.PolyLine(
    locations=[[observer_lat, observer_lon], [target_lat, target_lon]],
    color='green',
    weight=3
).add_to(m)

m.save("map.html")
print("Folium map saved to map.html")

# ===== CREATE PLOTLY 3D PLOT =====
fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=[x0], y=[y0], z=[z0],
    mode='markers',
    marker=dict(size=6, color='blue'),
    name='Camera'
))

fig.add_trace(go.Scatter3d(
    x=[ecef_target[0]], y=[ecef_target[1]], z=[ecef_target[2]],
    mode='markers',
    marker=dict(size=6, color='red'),
    name='Tank'
))

fig.add_trace(go.Scatter3d(
    x=[x0, ecef_target[0]],
    y=[y0, ecef_target[1]],
    z=[z0, ecef_target[2]],
    mode='lines',
    line=dict(color='green', width=3),
    name='Line of Sight'
))

fig.update_layout(
    title="3D ECEF Tank Localization",
    scene=dict(
        xaxis_title='X (m)',
        yaxis_title='Y (m)',
        zaxis_title='Z (m)',
        aspectmode='data'
    )
)

fig.write_html("plot.html", auto_open=False)
print("Plotly 3D plot saved to plot.html")
print("Open both map.html and plot.html in your browser to view results.")

Estimated tank position: lat=31.963230, lon=35.930444, alt=0.00
Folium map saved to map.html
Plotly 3D plot saved to plot.html
Open both map.html and plot.html in your browser to view results.


In [3]:
import numpy as np
import folium
import plotly.graph_objects as go
from pyproj import Transformer

# ===== INPUTS =====
observer_lat = 31.963158
observer_lon = 35.930359
observer_alt = 20  # meters

pitch_deg = -10   # camera pitch (negative = down)
yaw_deg = 45      # yaw degrees from North (0 = North, 90 = East)
target_distance = 100  # meters (max distance to tank)

# ===== COMPUTE DIRECTION VECTOR (ENU) =====
pitch = np.radians(pitch_deg)
yaw = np.radians(yaw_deg)

dx = np.cos(pitch) * np.sin(yaw)   # East component
dy = np.cos(pitch) * np.cos(yaw)   # North component
dz = np.sin(pitch)                  # Up component

direction_vector = np.array([dx, dy, dz])
direction_vector /= np.linalg.norm(direction_vector)

# Scale so that line intersects ground (up=0)
scale = -observer_alt / dz
scale = min(scale, target_distance)

enu_target = direction_vector * scale
east_offset, north_offset, up_offset = enu_target

# ===== COORDINATE TRANSFORMATIONS =====
ecef_from_llh = Transformer.from_crs("epsg:4326", "epsg:4978", always_xy=True)
llh_from_ecef = Transformer.from_crs("epsg:4978", "epsg:4326", always_xy=True)

x0, y0, z0 = ecef_from_llh.transform(observer_lon, observer_lat, observer_alt)
origin_ecef = np.array([x0, y0, z0])

lat_rad = np.radians(observer_lat)
lon_rad = np.radians(observer_lon)

east = np.array([-np.sin(lon_rad), np.cos(lon_rad), 0])
north = np.array([
    -np.sin(lat_rad)*np.cos(lon_rad),
    -np.sin(lat_rad)*np.sin(lon_rad),
    np.cos(lat_rad)
])
up = np.array([
    np.cos(lat_rad)*np.cos(lon_rad),
    np.cos(lat_rad)*np.sin(lon_rad),
    np.sin(lat_rad)
])

ecef_target = origin_ecef + east_offset*east + north_offset*north + up_offset*up
# Convert to lat/lon/alt first
target_lon, target_lat, _ = llh_from_ecef.transform(*ecef_target)

# Force tank altitude to zero (ground level)
target_alt = 0  

# Convert back to ECEF with ground altitude
ecef_target = np.array(ecef_from_llh.transform(target_lon, target_lat, target_alt))

print(f"Estimated tank position: lat={target_lat:.6f}, lon={target_lon:.6f}, alt={target_alt:.2f}")

# ===== CREATE FOLIUM MAP =====
m = folium.Map(location=[observer_lat, observer_lon], zoom_start=17)

folium.Marker(
    [observer_lat, observer_lon],
    popup="Camera",
    icon=folium.Icon(color='blue')
).add_to(m)

folium.Marker(
    [target_lat, target_lon],
    popup="Tank",
    icon=folium.Icon(color='red')
).add_to(m)

folium.PolyLine(
    locations=[[observer_lat, observer_lon], [target_lat, target_lon]],
    color='green',
    weight=3
).add_to(m)

m.save("map.html")
print("Folium map saved to map.html")

# ===== CREATE PLOTLY 3D PLOT =====
fig = go.Figure()

# Add observer (camera)
fig.add_trace(go.Scatter3d(
    x=[x0], y=[y0], z=[z0],
    mode='markers',
    marker=dict(size=6, color='blue'),
    name='Camera'
))

# Add tank
fig.add_trace(go.Scatter3d(
    x=[ecef_target[0]], y=[ecef_target[1]], z=[ecef_target[2]],
    mode='markers',
    marker=dict(size=6, color='red'),
    name='Tank'
))

# Add line of sight
fig.add_trace(go.Scatter3d(
    x=[x0, ecef_target[0]],
    y=[y0, ecef_target[1]],
    z=[z0, ecef_target[2]],
    mode='lines',
    line=dict(color='lime', width=8),  # Thicker brighter line
    name='Line of Sight'
))

# Add ground plane for reference
x_range = np.linspace(x0 - 100, x0 + 100, 10)
y_range = np.linspace(y0 - 100, y0 + 100, 10)
X, Y = np.meshgrid(x_range, y_range)
Z = np.zeros_like(X)  # Ground plane at zero altitude

fig.add_trace(go.Surface(
    x=X, y=Y, z=Z,
    colorscale='gray',
    opacity=0.5,
    showscale=False,
    name='Ground Plane'
))

# Update layout for better visualization
fig.update_layout(
    title="3D ECEF Tank Localization",
    scene=dict(
        xaxis_title='X (m)',
        yaxis_title='Y (m)',
        zaxis_title='Z (m)',
        aspectmode='data',
        camera=dict(
            eye=dict(x=1.25, y=1.25, z=1.25)  # Adjust camera angle for better view
        )
    )
)

fig.write_html("plot.html", auto_open=False)
print("Plotly 3D plot saved to plot.html")
print("Open both map.html and plot.html in your browser to view results.")

Estimated tank position: lat=31.963786, lon=35.931096, alt=0.00
Folium map saved to map.html
Plotly 3D plot saved to plot.html
Open both map.html and plot.html in your browser to view results.


In [5]:
import numpy as np
import folium
import plotly.graph_objects as go
from pyproj import Transformer

# ===== INPUTS =====
observer_lat = 31.963158
observer_lon = 35.930359
observer_alt = 20  # meters

pitch_deg = -10   # camera pitch (negative = down)
yaw_deg = 45      # yaw degrees from North (0 = North, 90 = East)
target_distance = 100  # meters (max distance to tank)

# ===== COMPUTE DIRECTION VECTOR (ENU) =====
pitch = np.radians(pitch_deg)
yaw = np.radians(yaw_deg)

dx = np.cos(pitch) * np.sin(yaw)   # East component
dy = np.cos(pitch) * np.cos(yaw)   # North component
dz = np.sin(pitch)                  # Up component

direction_vector = np.array([dx, dy, dz])
direction_vector /= np.linalg.norm(direction_vector)

# Scale so that line intersects ground (up=0)
scale = -observer_alt / dz
scale = min(scale, target_distance)

enu_target = direction_vector * scale
east_offset, north_offset, up_offset = enu_target

# ===== COORDINATE TRANSFORMATIONS =====
ecef_from_llh = Transformer.from_crs("epsg:4326", "epsg:4978", always_xy=True)
llh_from_ecef = Transformer.from_crs("epsg:4978", "epsg:4326", always_xy=True)

x0, y0, z0 = ecef_from_llh.transform(observer_lon, observer_lat, observer_alt)
origin_ecef = np.array([x0, y0, z0])

lat_rad = np.radians(observer_lat)
lon_rad = np.radians(observer_lon)

east = np.array([-np.sin(lon_rad), np.cos(lon_rad), 0])
north = np.array([
    -np.sin(lat_rad)*np.cos(lon_rad),
    -np.sin(lat_rad)*np.sin(lon_rad),
    np.cos(lat_rad)
])
up = np.array([
    np.cos(lat_rad)*np.cos(lon_rad),
    np.cos(lat_rad)*np.sin(lon_rad),
    np.sin(lat_rad)
])

ecef_target = origin_ecef + east_offset*east + north_offset*north + up_offset*up
target_lon, target_lat, target_alt = llh_from_ecef.transform(*ecef_target)

print(f"Estimated tank position: lat={target_lat:.6f}, lon={target_lon:.6f}, alt={target_alt:.2f}")

# ===== CREATE FOLIUM MAP =====
m = folium.Map(location=[observer_lat, observer_lon], zoom_start=17)

folium.Marker(
    [observer_lat, observer_lon],
    popup="Camera",
    icon=folium.Icon(color='blue')
).add_to(m)

folium.Marker(
    [target_lat, target_lon],
    popup="Tank",
    icon=folium.Icon(color='red')
).add_to(m)

folium.PolyLine(
    locations=[[observer_lat, observer_lon], [target_lat, target_lon]],
    color='green',
    weight=3
).add_to(m)

m.save("map.html")
print("Folium map saved to map.html")

# ===== CREATE PLOTLY 3D PLOT =====
fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=[x0], y=[y0], z=[z0],
    mode='markers',
    marker=dict(size=6, color='blue'),
    name='Camera'
))

fig.add_trace(go.Scatter3d(
    x=[ecef_target[0]], y=[ecef_target[1]], z=[ecef_target[2]],
    mode='markers',
    marker=dict(size=6, color='red'),
    name='Tank'
))

fig.add_trace(go.Scatter3d(
    x=[x0, ecef_target[0]],
    y=[y0, ecef_target[1]],
    z=[z0, ecef_target[2]],
    mode='lines',
    line=dict(color='green', width=3),
    name='Line of Sight'
))

fig.update_layout(
    title="3D ECEF Tank Localization",
    scene=dict(
        xaxis_title='X (m)',
        yaxis_title='Y (m)',
        zaxis_title='Z (m)',
        aspectmode='data'
    )
)

fig.write_html("plot.html", auto_open=False)
print("Plotly 3D plot saved to plot.html")
print("Open both map.html and plot.html in your browser to view results.")

Estimated tank position: lat=31.963786, lon=35.931096, alt=2.64
Folium map saved to map.html
Plotly 3D plot saved to plot.html
Open both map.html and plot.html in your browser to view results.
